# Выбор локации для скважины

**Описание проекта**

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 

**Цели и задачи**

- Gостроить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.
- Проанализировать возможную прибыль и риски техникой Bootstrap.

**Шаги для выбора локации**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**

- Данные геологоразведки трёх регионов находятся в файлах: geo_data_0.csv, geo_data_1.csv, geo_data_2.csv
- id — уникальный идентификатор скважины.
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы).
- product — объём запасов в скважине (тыс. баррелей).

**Условия задачи**
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**План выполнения работы**

[1  Загрузка и подготовка данных](#section1)

[1.1  Загрузка данных](#section1.1)

[1.2  Погдотовка данных](#section1.2)

[2  Обучение и проверка модели](#section2)

[2.1  Разбитие данных на обучающую и валидационную выборки в соотношении 75:25](#section2.1)

   - [2.1.1  обучающий/валидационный набор для первого региона](#section2.1.1)

   - [2.1.2  обучающий/валидационный набор для второго региона](#section2.1.2)

   - [2.1.3  обучающий/валидационный набор для третьего региона](#section2.1.3)
   

[2.2  Обучение/предсказание моделей](#section2.2)


[2.3  Результат предсказаний для всех регионов](#section2.3)

[3  Подготовка к расчёту прибыли](#section3)

[4  Расчёт прибыли и рисков](#section4)

[4.1  Расчёт прибыли](#section4.1)

[4.2  Расчёт рисков](#section4.2)

[5  Общий вывод](#section5)


<a id='section1'></a>
## Загрузка и подготовка данных

<a id='section1.1'></a>
### Загрузка данных

In [1]:
# добавляем необходимые библиотеки для работы
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from numpy import sqrt

In [2]:
# сохраним данные геологоразведки в переменных
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
# посмотрим на на имеющиеся таблицы, общий вид, информацию и проверим на наличие дубликатов
display(df_0.head())
print(df_0.info())
print(df_0.describe())
print()
print(f'количество дубликатов: {df_0.duplicated().sum()}')

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.3437

In [4]:
display(df_1.head())
print(df_1.info())
print(df_1.describe())
print()
print(f'количество дубликатов: {df_1.duplicated().sum()}')

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.7340

In [5]:
display(df_2.head())
print(df_2.info())
print(df_2.describe())
print()
print(f'количество дубликатов: {df_2.duplicated().sum()}')

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.8448

**Вывод:**
- пропусков нет
- дубликатов нет
- название столбцов корректное
- типы данных соответствуют описанию
- id — уникальный идентификатор скважины - для обучение не нужен
- features: f0, f1, f2
- target: product

<a id='section1.2'></a>
### Погдотовка данных

In [6]:
# Разбиваем на параметры и целевой признак для каждого из регионов
features_0 = df_0[['f0','f1','f2']]
target_0 = df_0[['product']]

features_1 = df_1[['f0','f1','f2']]
target_1 = df_1[['product']]

features_2 = df_2[['f0','f1','f2']]
target_2 = df_2[['product']]


In [7]:
# проверим результат
print(features_0.info())
print(target_0.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   f0      100000 non-null  float64
 1   f1      100000 non-null  float64
 2   f2      100000 non-null  float64
dtypes: float64(3)
memory usage: 2.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   product  100000 non-null  float64
dtypes: float64(1)
memory usage: 781.4 KB
None


In [8]:
print(features_1.info())
print(target_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   f0      100000 non-null  float64
 1   f1      100000 non-null  float64
 2   f2      100000 non-null  float64
dtypes: float64(3)
memory usage: 2.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   product  100000 non-null  float64
dtypes: float64(1)
memory usage: 781.4 KB
None


In [9]:
print(features_2.info())
print(target_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   f0      100000 non-null  float64
 1   f1      100000 non-null  float64
 2   f2      100000 non-null  float64
dtypes: float64(3)
memory usage: 2.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   product  100000 non-null  float64
dtypes: float64(1)
memory usage: 781.4 KB
None


In [10]:
# выведем значение random_state в константу для обращения к ней в последующем коде
state = 12345

<a id='section2'></a>
## Обучение и проверка модели

<a id='section2.1'></a>
### Разбитие данных на обучающую и валидационную выборки в соотношении 75:25

In [11]:
# функуия для создания обучающей и валидационной выборки
def create_function(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                                                  train_size = 0.75,
                                                                                  random_state = state,
                                                                                  shuffle = False)
    return features_train, features_valid, target_train, target_valid

<a id='section2.1.1'></a>
#### обучающий/валидационный набор для первого региона

In [12]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = create_function(features = features_0,
                                                                                 target = target_0)

In [13]:
print(features_train_0.shape)
print(target_train_0.shape)
print(features_valid_0.shape)
print(target_valid_0.shape)

(75000, 3)
(75000, 1)
(25000, 3)
(25000, 1)


<a id='section2.1.2'></a>
#### обучающий/валидационный набор для второго региона

In [14]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = create_function(features = features_1,
                                                                                 target = target_1)

In [15]:
print(features_train_1.shape)
print(target_train_1.shape)
print(features_valid_1.shape)
print(target_valid_1.shape)

(75000, 3)
(75000, 1)
(25000, 3)
(25000, 1)


<a id='section2.1.3'></a>
#### обучающий/валидационный набор для третьего региона

In [16]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = create_function(features = features_2,
                                                                                 target = target_2)

In [17]:
print(features_train_2.shape)
print(target_train_2.shape)
print(features_valid_2.shape)
print(target_valid_2.shape)

(75000, 3)
(75000, 1)
(25000, 3)
(25000, 1)


<a id='section2.2'></a>
### Обучение/предсказание моделей

In [18]:
# функция обучения/предсказания модели
def learning_function(features_train, target_train, features_valid, target_valid):
    scaler = StandardScaler()

    scaler.fit(features_train)

    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)

    model = LinearRegression()
    model.fit(features_train, target_train)

    predicted_valid = model.predict(features_valid)
    return model.score(features_valid, target_valid), predicted_valid

**Предсказание для первого региона**

In [19]:
model_score_0, predicted_valid_0 = learning_function(features_train = features_train_0,
                                                 target_train = target_train_0,
                                                 features_valid = features_valid_0,
                                                 target_valid = target_valid_0)

**Предсказание для второго региона**

In [20]:
model_score_1, predicted_valid_1 = learning_function(features_train = features_train_1,
                                                 target_train = target_train_1,
                                                 features_valid = features_valid_1,
                                                 target_valid = target_valid_1)

**Предсказание для третьего региона**

In [21]:
model_score_2, predicted_valid_2 = learning_function(features_train = features_train_2,
                                                 target_train = target_train_2,
                                                 features_valid = features_valid_2,
                                                 target_valid = target_valid_2)

<a id='section2.3'></a>
### Результат предсказаний для всех регионов

In [22]:
target_predict_0 = pd.concat([target_valid_0.reset_index(drop = True), pd.DataFrame(predicted_valid_0)], axis = 1)
target_predict_1 = pd.concat([target_valid_1.reset_index(drop = True), pd.DataFrame(predicted_valid_1)], axis = 1)
target_predict_2 = pd.concat([target_valid_2.reset_index(drop = True), pd.DataFrame(predicted_valid_2)], axis = 1)

target_predict_0 = target_predict_0.rename(columns = {0:'predicted_product'})
target_predict_1 = target_predict_1.rename(columns = {0:'predicted_product'})
target_predict_2 = target_predict_2.rename(columns = {0:'predicted_product'})

In [23]:
# объеденим в общую таблицу 
df_results = pd.DataFrame({'volume_of_materials'        :[target_valid_0['product'].sum(), # всего материала
                                                               target_valid_1['product'].sum(),
                                                               target_valid_2['product'].sum()],
                              'volume_forecast'      : [predicted_valid_0.sum(), # пронозируемый объём 
                                                                predicted_valid_1.sum(),
                                                                predicted_valid_2.sum()],
                              'average_forecast_volume' : [predicted_valid_0.mean(), # средний прогнозируемый объем материала
                                                                predicted_valid_1.mean(),
                                                                predicted_valid_2.mean()],
                              'RMSE'                         : [sqrt(mean_squared_error(target_valid_0, predicted_valid_0)),
                                                                sqrt(mean_squared_error(target_valid_1, predicted_valid_1)),
                                                                sqrt(mean_squared_error(target_valid_2, predicted_valid_2))],
                              'R2-score'                     : [model_score_0,
                                                                model_score_1,
                                                                model_score_2]
}
)

display(df_results)

,volume_of_materials,volume_forecast,average_forecast_volume,RMSE,R2-score
0,2.310666e+06,2.314842e+06,92.593681,37.650563,0.273593
1,1.723502e+06,1.723938e+06,68.957534,0.894446,0.999622
2,2.372809e+06,2.381887e+06,95.275462,40.003681,0.202021


**Вывод:**
- минимальное значение RMSE было получено для второго региона
- максимальное качество имеет модель второго региона
- наиболее перспективные регионы по запасу сырья - первый и третий регионы

<a id='section3'></a>
## Подготовка к расчёту прибыли

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки

- Бюджет на разработку скважин в регионе — 10 млрд рублей

- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей

- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью

In [24]:
# бюджет на разработку
BUDGET = 10**10
# Доход с каждой единицы продукта
INCOME = 4.5*10**5

In [25]:
breakeven = BUDGET/INCOME
print('{:.1f} тыс. баррелей сырья необходимо для обеспечения безубыточности'.format(breakeven))

22222.2 тыс. баррелей сырья необходимо для обеспечения безубыточности


In [26]:
breakeven_1 = breakeven/200
print('{:.1f} тыс. баррелей сырья необходимо для обеспечения безубыточности c одной из 200 скважин взятых для разработки'
      .format(breakeven_1))

111.1 тыс. баррелей сырья необходимо для обеспечения безубыточности c одной из 200 скважин взятых для разработки


**Вывод:**
- 22222.2 тыс. баррелей сырья необходимо для обеспечения безубыточности
- Минимальный объём продукта для безубыточной разработки составляет 111.1. Данный показатель рассчитан при условии выбора 200 лучших скважин из 500 в выборке
- Данное значение превышает средние значения запасов скважин по всем регионам

<a id='section4'></a>
## Расчёт прибыли и рисков 

<a id='section4.1'></a>
### Расчёт прибыли

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

- Выберем скважины с максимальными значениями предсказаний. Количество скважин зависит от бюджета компании и стоимости разработки одной скважины. В нашем случае выберем случайно 500 точек и отберём 200 с максимальным показателем объёма сырья
- Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям
- Рассчитаем прибыль для полученного объёма сырья

In [27]:
target_valid_0 = pd.Series(target_valid_0['product']).reset_index(drop=True)
target_valid_1 = pd.Series(target_valid_1['product']).reset_index(drop=True)
target_valid_2 = pd.Series(target_valid_2['product']).reset_index(drop=True)

In [28]:
predicted_valid_0 = pd.Series(*predicted_valid_0.reshape(1,-1))
predicted_valid_1 = pd.Series(*predicted_valid_1.reshape(1,-1))
predicted_valid_2 = pd.Series(*predicted_valid_2.reshape(1,-1))

In [29]:
def show_predicted_income(region, top_wells_mean, volume, income):
    print(region)
    print('Средний запас сырья среди скважин с максимальным показателем: {:.1f}'.format(top_wells_mean))
    print('Суммарный целевой объём сырья: {:.1f}'.format(volume))
    print('Прибыль для полученного объёма сырья: {:.1f}'.format(income))

In [30]:
# Функция берёт выборку размера 500, выбирает 200 наивысших значений
# и считает 
# - среднее значение объёма сырья для выбранных скважин
# - суммарный целевой объём сырья
# - прибыль для полученного объёма сырья

def predicted_income(target, predicts, region, state, replace = False, show_res = True, return_res = False):
    sample_preds = predicts.sample(n = 500, 
                                   replace = replace, 
                                   random_state = state)
    
    top_preds = sample_preds.sort_values(ascending = False)[:200]
    top_targets = target[top_preds.index]
    top_wells_mean = top_targets.mean()
    volume = sum(top_targets)
    income = volume * INCOME - BUDGET
    if show_res:
        show_predicted_income(region, top_wells_mean, volume, income)
    if return_res:
        return income

In [31]:
predicted_income(target_valid_0, predicted_valid_0, '***** Первый регион *****', state)

***** Первый регион *****
Средний запас сырья среди скважин с максимальным показателем: 112.9
Суммарный целевой объём сырья: 22581.7
Прибыль для полученного объёма сырья: 161746408.4


In [32]:
predicted_income(target_valid_1, predicted_valid_1, '***** Второй регион *****', state)

***** Второй регион *****
Средний запас сырья среди скважин с максимальным показателем: 119.0
Суммарный целевой объём сырья: 23795.9
Прибыль для полученного объёма сырья: 708136675.6


In [33]:
predicted_income(target_valid_2, predicted_valid_2, '*****Третий регион *****', state)

*****Третий регион *****
Средний запас сырья среди скважин с максимальным показателем: 113.1
Суммарный целевой объём сырья: 22611.3
Прибыль для полученного объёма сырья: 175067490.3


<a id='section4.1'></a>
### Расчёт рисков

- С помощью техники Bootstrap найдём распределение прибыли (1000 выборок)
- Найдём среднюю прибыль, 95%-й доверительный интервал и расчитаем риск убытка (отрицательную прибыль)

In [34]:
def show_risks(region, income_mean, conf_int_left, conf_int_right, loss_rate):
    print('*****', region, '*****')
    print('Средняя прибыль {:.2f}'.format(income_mean))
    print('95% доверительный интервал: {:.2f} : {:.2f}'.format(conf_int_left, conf_int_right))
    print('Риск убытков: {:.2f} %'.format(loss_rate * 100))

In [35]:
# функция реализует технику Bootstrap, количество выборок = bootstrap_samples = 1000
# Функция берёт выборку размера 500, выбирает 200 наивысших значений
# Считает общую прибыль и сохраняет в список
# 
def risk_calc(target, predicts, region):
    bootstrap_samples = 1000
    alpha = 0.05
    incomes = []
    state = np.random.RandomState(12345)
    for _ in range(bootstrap_samples):
        income = predicted_income(target, predicts, region, state, replace = True, show_res = False, return_res = True)
        incomes.append(income)

    incomes = pd.Series(incomes)
    income_mean = incomes.mean()
    # 95% доверительный интервал (считается с помощью функции quantile)
    conf_int_left = incomes.quantile(alpha / 2)
    conf_int_right = incomes.quantile(1 - alpha / 2)
    
    loss_count = 0
    for inc in incomes :
        if inc < 0 :
            loss_count += 1
    loss_rate = loss_count / bootstrap_samples
    show_risks(region, income_mean, conf_int_left, conf_int_right, loss_rate)

In [36]:
risk_calc(target_valid_0, predicted_valid_0, 'Первый регион')

***** Первый регион *****
Средняя прибыль 394011456.02
95% доверительный интервал: -81754411.95 : 882384873.08
Риск убытков: 5.90 %


In [37]:
risk_calc(target_valid_1, predicted_valid_1, 'Второй регион')

***** Второй регион *****
Средняя прибыль 476964651.27
95% доверительный интервал: 77413262.46 : 868054001.58
Риск убытков: 1.30 %


In [38]:
risk_calc(target_valid_2, predicted_valid_2, 'Третий регион')

***** Третий регион *****
Средняя прибыль 383653173.51
95% доверительный интервал: -119815929.36 : 896347708.87
Риск убытков: 6.60 %


**Вывод:**
- Средний запас сырья с одной скважины превосходит необходимый объём равный 111.1,  и равен: 112.9 для первого региона, 119.0 для второго региона и 113.1 для третьего региона (при расчёте показателей из 200 наилучших из 500 скважин)
- При применении техники Bootstrap:
  - Второй регион оказался с максимальной оценкой средней прибыли 476964651.27
  - Только второй регион прогнозирует прибыльную разработку по доверительному интервалу 95 %
  - Только второй регион имеет минимальный риск убытков и составляет 1.3 %


<a id='section5'></a>
## Общий вывод

**При начальном построении модели и предсказаний:**
- В первом и третьем регионах достаточно высокий средний показатель запаса предсказанного сырья. Однако RMSE для данных регионов также высок (37.65 для первого и 40.00 для третьего). Это может свидетельствовать о неоднозначности показателя, неточности модели регрессии
- Во втором регионе средний показатель запаса предсказанного сырья составляет 68.95 (ниже остальных регионов), при этом RMSE в данном регионе гораздо ниже чем в первом и в третьем (0.89), что свидетельствует о точности предсказаний и качестве построенной модели
- Средние показатели предсказанного сырья для одной скважины ниже теоретически необходимых (111.11)

**При расчёте показателей из 200 наилучших из 500 скважин:**
- Средний запас сырья с одной скважины среди всех регионов превосходит минимально необходимый объём

**При применении техники Bootstrap:**
- Оценка средней прибыли максимальна для второго региона (476.96 млн.)
- Второй регион прогнозирует прибыльную разработку по 95% доверительному интервалу
- Риск убытков для второго региона 1.3 %

**Рекомендуемый регион для разработки - второй**